# The `instrument` package for Bluesky Data Acquisition

*APS Training for Bluesky Data Acquisition*.

**Objective**

In this notebook, we describe the `instrument` package for a simulated X-ray instrument at a user facility such as the Advanced Photon Source.  The goal is to use EPICS as much as possible to provide the control system features and to use Bluesky (providing the data acquisition framework) as a thin layer on top of EPICS.

**Contents**

* [Start the `instrument` package](#start-the-instrument-package)
* [Description](#Description)
  * [`adsimdet`](#adsimdet)
  * [`scaler1`](#scaler1)
  * [`temperature`](#temperature)
  * [motors](#motors)
  * [`noisy`](#noisy)
  * [plans](#plans)
* [Log files](#log-files)
* [SPEC data files](#spec-data-files)
* [User code file](#user-code-file)
* [`databroker`](#databroker)

**Overview**

This simulated instrument is provided using [docker](https://www.docker.com/) images
for [EPICS base](https://www.aps.anl.gov/epics), the [synApps](https://www.aps.anl.gov/BCDA/synApps) [xxx](https://github.com/epics-modules/xxx) module, and [EPICS area detector](https://areadetector.github.io/master/index.html) [ADSimDetector](https://areadetector.github.io/master/ADSimDetector/simDetector.html?highlight=adsimdetector).  The images are based on these software versions:

* Ubuntu 20.04
* EPICS base 7.0.3
* synApps 6.1
* area detector 3.7

Two EPICS IOCs are provided:

prefix | description | docker image | documentation
:--- | :--- | :--- | :---
`ad:` | area detector IOC | [`prjemian/synapps-6.1-ad-3.7`](https://hub.docker.com/r/prjemian/synapps-6.1-ad-3.7) | https://github.com/prjemian/epics-docker/tree/main/n4_areaDetector
`gp:` | general purpose IOC | [`prjemian/synApps-6.1`](https://hub.docker.com/r/prjemian/synapps-6.1) | https://github.com/prjemian/epics-docker/tree/main/n3_synApps

## Start the `instrument` package

In [1]:
from instrument.collection import *

I Sun-10:32:41 - ############################################################ startup
I Sun-10:32:41 - logging started
I Sun-10:32:41 - logging level = 10
I Sun-10:32:41 - /home/apsu/bluesky/instrument/collection.py
I Sun-10:32:41 - /home/apsu/bluesky/instrument/mpl/notebook.py


Activating auto-logging. Current session state plus future input saved.
Filename       : /home/apsu/bluesky/.logs/ipython_console.log
Mode           : rotate
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


I Sun-10:32:41 - /home/apsu/bluesky/instrument/iocs/check_iocs.py
I Sun-10:32:41 - EPICS IOCs ready...
I Sun-10:32:41 - EPICS registers IOC ready...
I Sun-10:32:41 - #### Bluesky Framework ####
I Sun-10:32:41 - /home/apsu/bluesky/instrument/framework/check_python.py
I Sun-10:32:41 - /home/apsu/bluesky/instrument/framework/check_bluesky.py
I Sun-10:32:41 - /home/apsu/bluesky/instrument/framework/initialize.py
I Sun-10:32:42 - using databroker catalog 'training'
I Sun-10:32:42 - /home/apsu/bluesky/instrument/framework/metadata.py
I Sun-10:32:42 - #### Devices ####
I Sun-10:32:42 - /home/apsu/bluesky/instrument/devices/area_detector.py
I Sun-10:32:42 - /home/apsu/bluesky/instrument/devices/calculation_records.py
I Sun-10:32:55 - /home/apsu/bluesky/instrument/devices/ioc_stats.py
I Sun-10:32:55 - /home/apsu/bluesky/instrument/devices/motors.py
I Sun-10:32:56 - /home/apsu/bluesky/instrument/devices/noisy_detector.py
I Sun-10:32:56 - /home/apsu/bluesky/instrument/devices/scaler.py
I Sun-10:3

## Description

Might be a good idea to know now what this instrument package provides.  

Notably, the table includes:

ophyd name(s) | Description | label(s)
:--- | :---| :---
`adsimdet` | simulated EPICS area detector | `area_detector`
`calcs` & `calcouts` | calculation support |
`gp_stats` | details about the general purpose IOC |
`I0`, `diode`, ... | named scaler channels | `counter`
`m1` .. `m16` | 16 simulated EPICS motors | `motor`
`noisy` | simulated diffraction peak |
`scaler1` | simulated 16-channel EPICS scaler | `scalers`
`shutter` | simulated shutter | 
`temperature` | simulated temperature controller |

### `adsimdet`

The EPICS Area Detector ADSimDetector is included with this instrument as `adsimdet`.  The detector is a monochrome 1k x 1k frame.  The image is written to an HDF5 file and made available to the [`databroker`](#databroker) via a shared volume from the docker container.  `adsimdet` can be used as a detector.

The image is a simulated diffraction spot with center randomly-chosen between 100..900 on both axes.  The width is also random as is the peak intensity and noise level.  Furthermore, to simulate realistic conditions, the center position of both axes is adjusted by a few pixels using a pair of *swait* records (`gp:userCalc9` & `gp:userCalc10`) updating from random numbers.

In [2]:
adsimdet.summary()

data keys (* hints)
-------------------

read attrs
----------
hdf1                 MyHDF5Plugin        ('adsimdet_hdf1')

config keys
-----------
adsimdet_cam_acquire_period
adsimdet_cam_acquire_time
adsimdet_cam_image_mode
adsimdet_cam_manufacturer
adsimdet_cam_model
adsimdet_cam_num_exposures
adsimdet_cam_num_images
adsimdet_cam_trigger_mode

configuration attrs
-------------------
cam                  MyFixedCam          ('adsimdet_cam')
cam.acquire_period   EpicsSignalWithRBV  ('adsimdet_cam_acquire_period')
cam.acquire_time     EpicsSignalWithRBV  ('adsimdet_cam_acquire_time')
cam.image_mode       EpicsSignalWithRBV  ('adsimdet_cam_image_mode')
cam.manufacturer     EpicsSignalRO       ('adsimdet_cam_manufacturer')
cam.model            EpicsSignalRO       ('adsimdet_cam_model')
cam.num_exposures    EpicsSignalWithRBV  ('adsimdet_cam_num_exposures')
cam.num_images       EpicsSignalWithRBV  ('adsimdet_cam_num_images')
cam.trigger_mode     EpicsSignalWithRBV  ('adsimdet_cam_trigger_m

### `scaler1`

A simulated 16-channel scaler is configured with several channels of pulse counters, typical of a synchrotron beam line.  `scaler1` can be used as a detector.

In [3]:
scaler1.summary()

data keys (* hints)
-------------------
*I0
*diode
*roi1
*roi2
 scaler1_time
*scint
*timebase

read attrs
----------
channels             Channels            ('scaler1_channels')
channels.chan01      ScalerChannel       ('scaler1_channels_chan01')
channels.chan01.s    EpicsSignalRO       ('timebase')
channels.chan02      ScalerChannel       ('scaler1_channels_chan02')
channels.chan02.s    EpicsSignalRO       ('I0')
channels.chan03      ScalerChannel       ('scaler1_channels_chan03')
channels.chan03.s    EpicsSignalRO       ('scint')
channels.chan04      ScalerChannel       ('scaler1_channels_chan04')
channels.chan04.s    EpicsSignalRO       ('diode')
channels.chan11      ScalerChannel       ('scaler1_channels_chan11')
channels.chan11.s    EpicsSignalRO       ('roi1')
channels.chan12      ScalerChannel       ('scaler1_channels_chan12')
channels.chan12.s    EpicsSignalRO       ('roi2')
time                 EpicsSignal         ('scaler1_time')

config keys
-----------
scaler1_auto_count_d

The timebase is always shown.  The additional channels (using names from the scaler GUI screen) are shown output of `listdevice(scaler1)` below.

Note that in the instrument package, the channel names were assigned on startup.  An operating beam line would not do define the names here but instead, let the instrument team name these channels in the GUI screen at the time the cables from the pulse detectors are connected physically to the scaler.

Count the scaler from the command line:

In [4]:
%ct scalers

[This data will not be saved. Use the RunEngine to collect data.]
timebase                       6000000.0
I0                             3.0
scint                          4.0
diode                          3.0
roi1                           0.0
roi2                           0.0
scaler1_time                   0.6


NOTE: the actual counts are random numbers.  Also, the EPICS soft scaler seems to add 0.1s to the counting time.  Might be a bug with the EPICS scaler code.

In [5]:
listdevice(scaler1)

============ ========= ==========================
name         value     timestamp                 
============ ========= ==========================
timebase     6000000.0 2021-04-11 10:33:03.268842
I0           3.0       2021-04-11 10:33:03.268842
scint        4.0       2021-04-11 10:33:03.268842
diode        3.0       2021-04-11 10:33:03.268842
roi1         0.0       2021-04-11 10:33:03.268842
roi2         0.0       2021-04-11 10:33:03.268842
scaler1_time 0.6       2021-04-11 10:29:25.111676
============ ========= ==========================



As an added convenience, shortcut names for these channels were also assigned to local Python symbols (since the object path to the channels is easy to forget) in `instrument/devices/scaler.py` with these lines:

```python
timebase = scaler1.channels.chan01.s
I0 = scaler1.channels.chan02.s
scint = scaler1.channels.chan03.s
diode = scaler1.channels.chan04.s
```

This command: `scaler1.select_channels()` configures `scaler` to use just the channels with non-empty names.  To limit data collection to a subset of these channels, name the channels in a list argument, such as: `scaler1.select_channels(['I0', 'diode'])`

### `temperature`

The temperature simulator, implemented in `gp:userCalc8` works like a temperature controller, with setpoint and readback values.    `temperature` can be used as a detector **and** as a positioner (like a motor).

Additional controls for noise, update interval, maximum change per update, in-position tolerance, and done are provided, either through EPICS or in Python.  Once a new setpoint is entered, the readback will progress towards it, limited by the max change and the update interval.  Once at the setpoint, noise continues to be added to the temperature to provide realism.

In [6]:
temperature.summary()
listdevice(temperature)

data keys (* hints)
-------------------
*temperature
*temperature_setpoint

read attrs
----------
setpoint             EpicsSignal         ('temperature_setpoint')
readback             EpicsSignal         ('temperature')

config keys
-----------
temperature_calculation
temperature_description
temperature_max_change
temperature_noise
temperature_previous_value_pv
temperature_scanning_rate
temperature_tolerance

configuration attrs
-------------------
calculation          EpicsSignal         ('temperature_calculation')
description          EpicsSignal         ('temperature_description')
max_change           EpicsSignal         ('temperature_max_change')
noise                EpicsSignal         ('temperature_noise')
previous_value_pv    EpicsSignal         ('temperature_previous_value_pv')
scanning_rate        EpicsSignal         ('temperature_scanning_rate')
tolerance            EpicsSignal         ('temperature_tolerance')

unused attrs
------------
done                 Signal          

### motors

There are 16 soft motor channels: `gp:m1` .. `gp:m16`.  Any motor can be used as a detector **and** as a positioner.

The first motor, `gp:m1`, is used to compute a simulated 1-D diffraction peak ([`noisy`](#noisy)) using `gp:userCalc1`.

To make it easier to change the motor step size, a custom `MyEpicsMotor` class was made, subclassing from `ophyd.EpicsMotor` and adding a component to access the motor's `.SREV` (steps/revolution) field.  The default motor step size is 0.01 (with SREV=200).  For only `m1`, the step size has been changed to 0.001 by setting SREV=2000.

NOTE: Changing SREV (in this simulator* seems to affect the soft motor's simulated hardware limit.  Be careful if you adjust SREV to higher numbers (smaller step sizes) that the expected range of motion remains sufficient for your measurements.  Again, another thing to change in the EPICS support.

In [7]:
m1.summary()
listdevice(m1)

data keys (* hints)
-------------------
*m1
 m1_user_setpoint

read attrs
----------
user_readback        EpicsSignalRO       ('m1')
user_setpoint        EpicsSignal         ('m1_user_setpoint')

config keys
-----------
m1_acceleration
m1_motor_egu
m1_user_offset
m1_user_offset_dir
m1_velocity

configuration attrs
-------------------
user_offset          EpicsSignal         ('m1_user_offset')
user_offset_dir      EpicsSignal         ('m1_user_offset_dir')
velocity             EpicsSignal         ('m1_velocity')
acceleration         EpicsSignal         ('m1_acceleration')
motor_egu            EpicsSignal         ('m1_motor_egu')

unused attrs
------------
offset_freeze_switch EpicsSignal         ('m1_offset_freeze_switch')
set_use_switch       EpicsSignal         ('m1_set_use_switch')
motor_is_moving      EpicsSignalRO       ('m1_motor_is_moving')
motor_done_move      EpicsSignalRO       ('m1_motor_done_move')
high_limit_switch    EpicsSignalRO       ('m1_high_limit_switch')
low_limit_s

### `noisy`

A simulated diffraction peak is computed in 1-D using a Lorentzian function in EPICS PV `gp:userCalc1`.  The simulation uses the position of motor `gp:m1` and random choices for center, width, noise, and peak intensity (scale factor).  `noisy` can be used as a detector.

NOTE: `noisy` is an `ophyd.EpicsSignal` (variant), thus it lacks the `.summary()` method that `ophyd.Device` objects have.

In [8]:
listdevice(noisy)

===== ================= ==========================
name  value             timestamp                 
===== ================= ==========================
noisy 7291.702008410923 2021-04-11 10:32:56.973880
===== ================= ==========================



### plans

Several examples of user plan are provided in file `instrument/plans/peak_finder_example.py`.  These plans are described and used in the notebook [Lineup a 1-D peak](https://nbviewer.jupyter.org/github/BCDA-APS/bluesky_instrument_training/blob/main/lineup_1d_peak.ipynb).

## Log files

For diagnostic and general support, log files are created to record activity.  In the working directory, the log files are written to a `./.logs` subdirectory.

There are two kinds of file, one that records user commands and the python
result, the other records items sent to the
[customized](https://github.com/prjemian/stdlogpj#example-directing-logs-to-a-specific-directory)
Python [logging](https://docs.python.org/3/library/logging.html) package.

In the IPython session, use the `!` to run a linux command:

In [9]:
!ls -lAFgh .logs

total 172K
-rw-rw-r-- 1 apsu  707 Apr 11 10:33 ipython_console.log
-rw-rw-r-- 1 apsu  548 Apr 11 10:32 ipython_console.log.001~
-rw-rw-r-- 1 apsu 1.1K Apr 11 10:31 ipython_console.log.002~
-rw-rw-r-- 1 apsu 3.2K Apr 11 10:28 ipython_console.log.003~
-rw-rw-r-- 1 apsu 2.0K Apr 11 10:24 ipython_console.log.004~
-rw-rw-r-- 1 apsu  238 Apr 11 10:15 ipython_console.log.005~
-rw-rw-r-- 1 apsu   55 Apr 11 10:14 ipython_console.log.006~
-rw-rw-r-- 1 apsu 3.5K Apr 11 09:58 ipython_console.log.007~
-rw-rw-r-- 1 apsu 4.1K Apr 11 10:14 ipython_console.log.008~
-rw-rw-r-- 1 apsu  136 Apr 11 09:21 ipython_console.log.009~
-rw-rw-r-- 1 apsu  187 Apr 11 09:15 ipython_console.log.010~
-rw-rw-r-- 1 apsu 1.3K Apr 11 09:08 ipython_console.log.011~
-rw-rw-r-- 1 apsu 2.3K Apr 11 09:05 ipython_console.log.012~
-rw-rw-r-- 1 apsu 1.6K Apr 11 09:04 ipython_console.log.013~
-rw-rw-r-- 1 apsu   20 Apr 11 09:02 ipython_console.log.014~
-rw-rw-r-- 1 apsu  345 Apr 10 21:37 ipython_console.log.015~
-rw-rw-r-- 1 apsu 

Let's take a look at a few lines of each type of file, to get a feel for the information logged.

The `ipython_console.log` file is created for every session (every time IPython is started or every time the Jupyter kernel is started).  It records the commands that were entered and the output, if any, from each command.  It does not record anything that was sent to the console by `print()` statements.  The older log files are numbered, higher number is older (each new session shifts these numbers by 1).

In [10]:
!head .logs/ipython_console.log

# IPython log file

# Sun, 11 Apr 2021 10:33:01
adsimdet.summary()
# Sun, 11 Apr 2021 10:33:02
scaler1.summary()
# Sun, 11 Apr 2021 10:33:02
get_ipython().run_line_magic('ct', 'scalers')
# Sun, 11 Apr 2021 10:33:03
listdevice(scaler1)


The `ipython_logger.log` file(s) contain the output from calls to the `logger`.  The files are appended with new logger reports until the file reaches ca. 1 MB.  (Larger files are slow to append.)  Then the file is given a number and a new logger file is created.  Higher numbers are older.  At most, 9 numbered files are retained (to avoid filling disk storage with unnecessary diagnostics).

A single *logger* file may contain reports from several sessions.  The third piece of information is the `pid` (process identifier) of the session.  The PID is assigned by the operating system when the session is started.

The logger file contains a full report while the report's representation is more terse.  (Users do not always want the full details.  Just remember they are available in the logger file.)  Here's the last few logger lines as shown on the console:

    I Wed-00:24:52 - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/devices/temperature_signal.py
    I Wed-00:24:52 - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/plans/peak_finder_example.py
    I Wed-00:24:52 - Startup is complete.

Those same lines appear in the logger file as:

    |2021-02-24 00:24:52.091|INFO|25986|bluesky-session|temperature_signal|11|MainThread| - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/devices/temperature_signal.py
    |2021-02-24 00:24:52.165|INFO|25986|bluesky-session|peak_finder_example|15|MainThread| - /home/mintadmin/Documents/projects/BCDA-APS/bluesky_instrument_training/instrument/plans/peak_finder_example.py
    |2021-02-24 00:24:52.179|INFO|25986|bluesky-session|collection|23|MainThread| - Startup is complete.


In [11]:
!tail .logs/ipython_logger.log

|2021-04-11 10:33:01.328|INFO|24237|bluesky-session|spec_data_file_writer|13|MainThread| - /home/apsu/bluesky/instrument/callbacks/spec_data_file_writer.py
|2021-04-11 10:33:01.329|INFO|24237|bluesky-session|spec_data_file_writer|29|MainThread| - writing to SPEC file: /home/apsu/bluesky/20210411-103301.dat
|2021-04-11 10:33:01.331|INFO|24237|bluesky-session|spec_data_file_writer|30|MainThread| -    >>>>   Using default SPEC file name   <<<<
|2021-04-11 10:33:01.336|INFO|24237|bluesky-session|spec_data_file_writer|31|MainThread| -    file will be created when bluesky ends its next scan
|2021-04-11 10:33:01.337|INFO|24237|bluesky-session|spec_data_file_writer|32|MainThread| -    to change SPEC file, use command:   newSpecFile('title')
|2021-04-11 10:33:01.338|INFO|24237|bluesky-session|collection|21|MainThread| - #### Plans ####
|2021-04-11 10:33:01.339|INFO|24237|bluesky-session|peak_finder_example|15|MainThread| - /home/apsu/bluesky/instrument/plans/peak_finder_example.py
|2021-04-11 1

## SPEC data files

A [SPEC](https://certif.com) data file records a copy of the scan data (area detector images are not stored in SPEC files).  By default, the file is created in the present working directory using a name constructed from the date and time with a `.dat` file extension.  Unlike SPEC, new data is written *at the end of a scan*.  It is also possible to use the [`SpecWriterCallback`](https://apstools.readthedocs.io/en/latest/source/_filewriters.html#apstools.filewriters.SpecWriterCallback) to write data extracted from the database after the experiment is done.

The 2021-03 training does not cover the use of these files.  You are free to examine them yourselves.

<summary>Example SPEC data file from a bluesky session:</summary>

<details>


    #F /home/mintadmin/Documents/projects/BCDA-APS/bluesky_instrument_training/20210223-222553.dat
    #E 1614140753
    #D Tue Feb 23 22:25:53 2021
    #C Bluesky  user = mintadmin  host = mint-vm
    #O0 
    #o0 

    #S 216  count(detectors=['scaler1', 'noisy', 'temperature'], num=1)
    #D Tue Feb 23 22:25:59 2021
    #C Tue Feb 23 22:25:59 2021.  plan_type = generator
    #C Tue Feb 23 22:25:59 2021.  uid = 8c4f4b62-81ec-4fe3-bfa2-18178d37876b
    #MD uid = 8c4f4b62-81ec-4fe3-bfa2-18178d37876b
    #MD beamline_id = APS_Python_training_2021
    #MD detectors = ['scaler1', 'noisy', 'temperature']
    #MD instrument_name = class_2021_03
    #MD login_id = mintadmin@mint-vm
    #MD num_intervals = 0
    #MD num_points = 1
    #MD pid = 20559
    #MD proposal_id = training
    #MD versions = {'apstools': '1.4.0+1.g50c6f9d', 'bluesky': '1.6.7', 'databroker': '1.1.0', 'epics': '3.4.3', 'h5py': '2.10.0', 'intake': '0.6.0', 'matplotlib': '3.3.3', 'numpy': '1.18.5', 'ophyd': '1.6.0', 'pyRestTable': '2020.0.3', 'spec2nexus': '2021.1.8'}
    #P0 
    #N 20
    #L Epoch_float  Epoch  temperature_setpoint  temperature_readback  temperature_done  temperature_calculation  temperature_description  temperature_max_change  temperature_noise  temperature_previous_value_pv  temperature_scanning_rate  temperature_tolerance  timebase  I0  scint  diode  scaler1_time  scaler1  noisy  temperature
    2.859903573989868 3 25.0 25.35954070344091 True 0 0 2.0 1.0 0 5 1.0 21000000.0 10.0 9.0 9.0 2.1 0 29.61249046818531 0
    #U 0 temperature_calculation A+max(-D,min(D,(B-A)))+C*(RNDM-0.5)
    #U 0 temperature_description temperature
    #U 0 temperature_previous_value_pv gp:userCalc8.VAL
    #C Tue Feb 23 22:26:02 2021.  num_events_baseline = 2
    #C Tue Feb 23 22:26:02 2021.  num_events_primary = 1
    #C Tue Feb 23 22:26:02 2021.  exit_status = success

    #S 217  count(detectors=['scaler1', 'noisy', 'temperature'], num=5)
    #D Tue Feb 23 22:26:02 2021
    #C Tue Feb 23 22:26:02 2021.  plan_type = generator
    #C Tue Feb 23 22:26:02 2021.  uid = 1e857ae9-b5bf-4965-a709-b290fb04d30f
    #MD uid = 1e857ae9-b5bf-4965-a709-b290fb04d30f
    #MD beamline_id = APS_Python_training_2021
    #MD detectors = ['scaler1', 'noisy', 'temperature']
    #MD instrument_name = class_2021_03
    #MD login_id = mintadmin@mint-vm
    #MD num_intervals = 4
    #MD num_points = 5
    #MD pid = 20559
    #MD proposal_id = training
    #MD versions = {'apstools': '1.4.0+1.g50c6f9d', 'bluesky': '1.6.7', 'databroker': '1.1.0', 'epics': '3.4.3', 'h5py': '2.10.0', 'intake': '0.6.0', 'matplotlib': '3.3.3', 'numpy': '1.18.5', 'ophyd': '1.6.0', 'pyRestTable': '2020.0.3', 'spec2nexus': '2021.1.8'}
    #P0 
    #N 20
    #L Epoch_float  Epoch  temperature_setpoint  temperature_readback  temperature_done  temperature_calculation  temperature_description  temperature_max_change  temperature_noise  temperature_previous_value_pv  temperature_scanning_rate  temperature_tolerance  timebase  I0  scint  diode  scaler1_time  scaler1  noisy  temperature
    3.391828775405884 3 25.0 25.304470893415733 True 0 0 2.0 1.0 0 5 1.0 21000000.0 10.0 11.0 13.0 2.1 0 29.61249046818531 0
    #U 0 temperature_calculation A+max(-D,min(D,(B-A)))+C*(RNDM-0.5)
    #U 0 temperature_description temperature
    #U 0 temperature_previous_value_pv gp:userCalc8.VAL
    6.6996848583221436 7 25.0 24.56161593041886 True 1 1 2.0 1.0 1 5 1.0 21000000.0 11.0 10.0 11.0 2.1 0 29.61249046818531 0
    #U 1 temperature_calculation A+max(-D,min(D,(B-A)))+C*(RNDM-0.5)
    #U 1 temperature_description temperature
    #U 1 temperature_previous_value_pv gp:userCalc8.VAL
    9.815467119216919 10 25.0 24.70134279392691 True 2 2 2.0 1.0 2 5 1.0 21000000.0 10.0 11.0 8.0 2.1 0 29.61249046818531 0
    #U 2 temperature_calculation A+max(-D,min(D,(B-A)))+C*(RNDM-0.5)
    #U 2 temperature_description temperature
    #U 2 temperature_previous_value_pv gp:userCalc8.VAL
    12.774443626403809 13 25.0 25.29922178988327 True 3 3 2.0 1.0 3 5 1.0 21000000.0 11.0 11.0 11.0 2.1 0 29.61249046818531 0
    #U 3 temperature_calculation A+max(-D,min(D,(B-A)))+C*(RNDM-0.5)
    #U 3 temperature_description temperature
    #U 3 temperature_previous_value_pv gp:userCalc8.VAL
    16.023823261260986 16 25.0 25.317425803006028 True 4 4 2.0 1.0 4 5 1.0 21000000.0 9.0 9.0 10.0 2.1 0 29.61249046818531 0
    #U 4 temperature_calculation A+max(-D,min(D,(B-A)))+C*(RNDM-0.5)
    #U 4 temperature_description temperature
    #U 4 temperature_previous_value_pv gp:userCalc8.VAL
    #C Tue Feb 23 22:26:19 2021.  num_events_baseline = 2
    #C Tue Feb 23 22:26:19 2021.  num_events_primary = 5
    #C Tue Feb 23 22:26:19 2021.  exit_status = success

    #S 218  count(detectors=['temperature'], num=5)
    #D Tue Feb 23 22:26:21 2021
    #C Tue Feb 23 22:26:21 2021.  plan_type = generator
    #C Tue Feb 23 22:26:21 2021.  uid = b364f246-f058-43e3-b6b6-ff08003f1b63
    #MD uid = b364f246-f058-43e3-b6b6-ff08003f1b63
    #MD beamline_id = APS_Python_training_2021
    #MD detectors = ['temperature']
    #MD instrument_name = class_2021_03
    #MD login_id = mintadmin@mint-vm
    #MD num_intervals = 4
    #MD num_points = 5
    #MD pid = 20559
    #MD proposal_id = training
    #MD versions = {'apstools': '1.4.0+1.g50c6f9d', 'bluesky': '1.6.7', 'databroker': '1.1.0', 'epics': '3.4.3', 'h5py': '2.10.0', 'intake': '0.6.0', 'matplotlib': '3.3.3', 'numpy': '1.18.5', 'ophyd': '1.6.0', 'pyRestTable': '2020.0.3', 'spec2nexus': '2021.1.8'}
    #P0 
    #N 13
    #L Epoch_float  Epoch  temperature_setpoint  temperature_readback  temperature_done  temperature_calculation  temperature_description  temperature_max_change  temperature_noise  temperature_previous_value_pv  temperature_scanning_rate  temperature_tolerance  temperature
    0.7077298164367676 1 25.0 25.43418783855955 True 0 0 2.0 1.0 0 5 1.0 0
    #U 0 temperature_calculation A+max(-D,min(D,(B-A)))+C*(RNDM-0.5)
    #U 0 temperature_description temperature
    #U 0 temperature_previous_value_pv gp:userCalc8.VAL
    0.8045623302459717 1 25.0 25.43418783855955 True 1 1 2.0 1.0 1 5 1.0 0
    #U 1 temperature_calculation A+max(-D,min(D,(B-A)))+C*(RNDM-0.5)
    #U 1 temperature_description temperature
    #U 1 temperature_previous_value_pv gp:userCalc8.VAL
    0.8914773464202881 1 25.0 25.43418783855955 True 2 2 2.0 1.0 2 5 1.0 0
    #U 2 temperature_calculation A+max(-D,min(D,(B-A)))+C*(RNDM-0.5)
    #U 2 temperature_description temperature
    #U 2 temperature_previous_value_pv gp:userCalc8.VAL
    0.9808797836303711 1 25.0 25.43418783855955 True 3 3 2.0 1.0 3 5 1.0 0
    #U 3 temperature_calculation A+max(-D,min(D,(B-A)))+C*(RNDM-0.5)
    #U 3 temperature_description temperature
    #U 3 temperature_previous_value_pv gp:userCalc8.VAL
    1.0428459644317627 1 25.0 25.43418783855955 True 4 4 2.0 1.0 4 5 1.0 0
    #U 4 temperature_calculation A+max(-D,min(D,(B-A)))+C*(RNDM-0.5)
    #U 4 temperature_description temperature
    #U 4 temperature_previous_value_pv gp:userCalc8.VAL
    #C Tue Feb 23 22:26:22 2021.  num_events_baseline = 2
    #C Tue Feb 23 22:26:22 2021.  num_events_primary = 5
    #C Tue Feb 23 22:26:22 2021.  exit_status = success

</details>

## User Code File

A common request from instruments is to allow the user to write some python code that can be loaded into the current session.  Since this codedevelopment of this *user code* is often iterative, it must be possible to reload the code without requiring the session to exit and restart.

The local file [`pv_finder.py`](pv_finder.py) provides an example of such a user code file.  Load this file with the command:

    %run -im pv_finder

Note that since the `m` (as module) command option was added, it is not necessary to give the Python extension `.py`.

This file loads the command `findpv(PVNAME)` which identifies the name of the ophyd object(s) using the PV name passed as an argument.  Take for example, we want to find PV `gp:userCalc8.CALC`:

In [12]:
%run -im pv_finder

AttributeError: 'NoneType' object has no attribute 'origin'

In [ ]:
findpv("gp:userCalc8.CALC")

Observe that the same `.calculation` is both readable and writable from two different ophyd objects: `calcs.calc8` and `temperature`.

## databroker

The [`databroker`](https://blueskyproject.io/databroker) package provides 
a Python interface to the database with the experiment data, including 
references to the large file content such as area detector images.  A 
[YAML](https://yaml.org) configuration file connects databroker with a 
specific repository in the MongoDB database server.  In the example here 
([`bluesky_class.yml`](./bluesky_class.yml)), the name of the *catalog* entry
in this file is `class_2021_03`.  It makes two connections to a MongoDB server
running on the same workstation `localhost`.  Both connections are to the 
same MongoDB collection: `class_2021_03-bluesky`.  The name of the file
is not important as long as it is placed in a directory searched by 
`databroker.catalog`.

**Example**

```yml
# file: bluesky_class.yml
# purpose: Configuration file to connect Bluesky databroker with MongoDB
# For 2021-03 Python Training at APS

# Copy to: ~/.local/share/intake/bluesky_class.yml
# Create subdirectories as needed

sources:
  class_2021_03:
    args:
      asset_registry_db: mongodb://localhost:27017/class_2021_03-bluesky
      metadatastore_db: mongodb://localhost:27017/class_2021_03-bluesky
    driver: bluesky-mongo-normalized-catalog
```

When the bluesky session starts, this `class_2021_03` catalog is referenced when
creating the `db` object in the instrument package, in 
`instrument.framework.initialize.py` by these lines:

```python
catalog_name = "class_2021_03"
db = databroker.catalog[catalog_name].v1
logger.info(f"using databroker catalog '{catalog_name}'")
```